In [1]:
pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.5 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10766 sha256=a4608babb2e36c20b202e8f4525a4a3b9f6fa40c49f82afc69cb423fd294498d
  Stored in di

In [2]:
import pandas as pd
tables= pd.read_csv("tables_SSR.csv", sep=",",encoding='utf-8',na_values="")
tables.rename(columns={'Table correspondante':'tab','Séjour / ACE':'struct', 'Public / privé': 'champ','Observation CNAMTS': 'observation' }, inplace=True)

tablestruct={
    'tables':tables['tab'].to_list(),
    'structure':tables['struct'].to_list(),
    'champ': tables['champ'].to_list(), 
    'observation': tables['observation'].to_list()
}

In [5]:
import re
import json
from google.colab import files
import pandas as pd 
df= pd.read_csv("SSR.csv",sep=",",encoding='utf-8',na_values="")
df.rename(columns={'Nom variable':'variables','Observation CNAMTS':'observation','Libellé':'description','Longueur':'length', 'Année de création':'dateCreated', 'Année avant disparition':'dateDeleted','Séjour / ACE':'struct','Public / privé': 'champ',  'Description table':'tab_lib'}, inplace=True)
df=df.fillna('')



##### transformation des types 
mapping_t = {
    "Char": "string",
    "char":"string",
    "Num":"integer",
    "num":"integer",
    "date":"date",
    "Date":"date",
    "":"string" # cas où le type est vide
  }
df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
def numbint(x): 
  res = re.search(",", x)
  if res:
    return "number"
  return "integer"

def strdate(x):
  if 'MOI' in x:
    return "date"
  return "string"

df["datemissing"]=''

df['nomenclature']='-'
df["rdg"]=''

  


 # Creation dy type oracle 
mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des bons types
for i in range(len(df['variables'])):
  if df['type'][i] =='number':
    df['type'][i]=numbint(df['length'][i])  
  elif df['type'][i]=='string':
    df['type'][i]=strdate(df['variables'][i])

  

# create json file with the tables name
tableexist={
    'table':[],
    'tab_lib':[],
    'champ':[],
    'produit':'PMSI SSR'
}


for z in range(len(df['variables'])):
  if df['dateDeleted'][z] is not '':
    df['dateDeleted'][z]=df['dateDeleted'][z]+1

df['datereated']=''
for z in range(len(df['variables'])):
   # Transformation of type 
  if df['dateCreated'][z] is not '':
    df['dateCreated'][z] = int(float(df['dateCreated'][z]))
  if df['dateDeleted'][z] is not '':  
    df['dateDeleted'][z] = int (float(df['dateDeleted'][z]))
  df=df.fillna('')
  if df['Table'][z] in tableexist['table']: 
    
    new_dict={
              "name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
    
    with open('{}.json'.format(df['Table'][z]),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  else:
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump([
            {"name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]}
        ], f, indent=4, ensure_ascii=False)
      tableexist['table'].append(df['Table'][z])
      tableexist['tab_lib'].append(df['tab_lib'][z])
      tableexist['champ'].append(df['champ'][i])
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['table']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']

     # La Table des sejours SSR
  for tables in tablestruct['tables']:
    #séjours Public et Privé
    if tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=="Public et Privé" : 
      if table=='T_SSRaaB':
        schema.descriptor['primaryKey']= [
        "ETA_NUM",
        "RHA_NUM",
        "RHS_NUM"
        ]
        schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM"],
                                           "reference": {
                                             "resource": "T_SSRaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
      elif table=="T_SSRaaE": # Table des établissements
        schema.descriptor['primaryKey']= [
            "ETA_NUM"
            ]
 
     # On identifie un séjour par 2 clés de jointure et un RHA par 3 clés de jointure                                    
     

     #1 Tables dont la jointure avec la table principale B se fait par le Séjours : Résumé Hebdomadaires Standardisés Anonymisés
      elif table=='T_SSRaaC':
        schema.descriptor['primaryKey']= [
          "ETA_NUM",
          "RHA_NUM"
          ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                            "ETA_NUM",
                                            "RHA_NUM"
                                        ],
                                        "reference": {
                                              "resource": "T_SSRaaB",
                                              "fields": [
                                                  "ETA_NUM",
                                                  "RHA_NUM"
                                              ]
                                              },
                                        "description" : "Bidirectionnel"
                                        },
                                        {
                                            "fields": "NIR_ANO_17", 
                                            "reference": {
                                              "resource": "IR_BEN_R",
                                              "fields": "BEN_NIR_PSA"
                                        }
                                        }
                                        ,
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_IMB_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "ER_PRS_F",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          }]
      elif table in ['T_SSRaaGME', 'T_SSRaaS']:
        schema.descriptor['foreignKeys']=[{"fields":[
                                            "ETA_NUM",
                                            "RHA_NUM"
                                        ],
                                        "reference": {
                                              "resource": "T_SSRaaB",
                                              "fields": [
                                                  "ETA_NUM",
                                                  "RHA_NUM"
                                              ]
                                              }
                                        }
                                        ]
     #2 Tables dont la jointure avec la table principale B se fait par le RHA : Résumé Hebdomadaires Standardisés Anonymisés 

      elif table in ['T_SSRaaCCAM', 'T_SSRaaD', 'T_SSRaaCSAAR', 'T_SSRaaMED', 'T_SSRaaMEDATU', 'T_SSRaaLEG', 'T_SSRaaVALO', 'T_SSRaaCSARR']:
        schema.descriptor['foreignKeys']=[{"fields":[
                                            "ETA_NUM",
                                            "RHA_NUM",
                                            "RHS_NUM"
                                        ],
                                        "reference": {
                                              "resource": "T_SSRaaB",
                                              "fields": [
                                                  "ETA_NUM",
                                                  "RHA_NUM",
                                                  "RHS_NUM"
                                              ]
                                              }
                                        }
                                        ]      
      
        
      
      else:
        schema.descriptor['foreignKeys']=[
            {
                "fields": [
                    "ETA_NUM",
                    "RHA_NUM"

                ],
                "reference": {
                    "resource": "T_SSRaaB",
                    "fields": [
                        "ETA_NUM",
                        "RHA_NUM"
                    ]
                }
            }
        ]

   #séjours Public
    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=='Public' : 
      schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM",
                                              "RHA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_SSRaaB",
                                                "fields": [
                                                    "ETA_NUM",
                                                    "RHA_NUM"
                                                ]
                                                }
                                          }
                                          ]
    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=="Privé" : #séjours Privés: tables de facturation en établissement Privé
      if table=='T_SSRaaFA':
        schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "RHA_NUM"
            ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM",
                                              "RHA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_SSRaaC",
                                                "fields": [
                                                    "ETA_NUM",
                                                    "RHA_NUM"
                                                ]
                                                }
          }]
                                          
      else:
        schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM",
                                              "RHA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_SSRaaFA",
                                                "fields": [
                                                    "ETA_NUM",
                                                    "RHA_NUM"
                                                ]
                                                }
                                          }
                                          ]

                                                  # Tables des ACE 
    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='ACE':
      if table=="T_SSRaaCSTC":
        schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "SEQ_NUM"
            ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                                          "ETA_NUM",
                                                          "SEQ_NUM"
                                                          ], 
                                              "reference": {
                                                "resource": "T_SSRaaFASTC",
                                                "fields":[
                                                          "ETA_NUM",
                                                          "SEQ_NUM"
                                                          
                                                ]},
                                          "description" : "Bidirectionnel"
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_BEN_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_IMB_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "ER_PRS_F",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          }
                                          
                                        ]
      elif table=="T_SSRaaFASTC":
        schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "SEQ_NUM"
            ]
      else:
        schema.descriptor['foreignKeys']=[
            {
                "fields": [
                    "ETA_NUM",
                    "SEQ_NUM"

                ],
                "reference": {
                    "resource": "T_SSRaaFASTC",
                    "fields": [
                        "ETA_NUM",
                        "SEQ_NUM"
                    ]
                }
            }
        ]
        

        
      #les autres tables : séjours SSR et ACE non individualisés, sejours et ACE à la fois

    elif tablestruct['structure'][tablestruct['tables'].index(table)] in ['Séjour (non individualisé)', 'ACE (non individualisé)','Séjour et ACE'] :
      schema.descriptor['foreignKeys']= [{"fields":["ETA_NUM"],
                                              "reference": {
                                                "resource": "T_SSRaaE",
                                                "fields":["ETA_NUM"]
                                                }
                                        
                                              }
                                            
                                            ]

          

      
    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                          "dateDeleted": "",
                                          "dateMissing": []}
    schema.descriptor['champ']=tablestruct['champ'][tablestruct['tables'].index(table)]
    schema.descriptor['observation']=str(tablestruct['observation'][tablestruct['tables'].index(table)])                             
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)

    if table=='T_SSRaaB':
      schema.descriptor['primaryKey']= [
        "ETA_NUM",
        "RHA_NUM",
        "RHS_NUM"
        ]
      schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM"],
                                           "reference": {
                                             "resource": "T_SSRaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
  
    elif table=='T_SSRaaC':
      schema.descriptor['primaryKey']= [
        "ETA_NUM",
        "RHA_NUM"
        ]
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM",
                                          "RHA_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_SSRaaB",
                                             "fields": [
                                                "ETA_NUM",
                                                "RHA_NUM"
                                            ]
                                            },
                                       "description" : "Bidirectionnel"
                                       },
                                      {
                                          "fields": "NIR_ANO_17", 
                                          "reference": {
                                             "resource": "IR_BEN_R",
                                             "fields": "BEN_NIR_PSA"
                                      }
                                      }
                                      ]
    elif table in ['T_SSRaaGME', 'T_SSRaaS']:
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM",
                                          "RHA_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_SSRaaB",
                                             "fields": [
                                                "ETA_NUM",
                                                "RHA_NUM"
                                            ]
                                            }
                                       }
                                      ]
    elif table in ['T_SSRaaCCAM', 'T_SSRaaS']:
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM",
                                          "RHA_NUM",
                                          "RHS_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_SSRaaB",
                                             "fields": [
                                                "ETA_NUM",
                                                "RHA_NUM",
                                                "RHS_NUM"
                                            ]
                                            }
                                       }
                                      ]
    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.descriptor['champ']=tableexist['champ'][tableexist['table'].index(table)]
    schema.descriptor['observation']=""                               
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
from google.colab import files


<>:72: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:72: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-5-8708b044dfb2>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'][i]=strdate(df['variables'][i])
<ipython-input-5-8708b044dfb2>:72: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if df['dateDeleted'][z] is not '':
<ipython-input-5-8708b044dfb2>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [6]:
from google.colab import files
!zip -r /content/sample_data.zip /content


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.01.04/ (stored 0%)
  adding: content/.config/logs/2023.01.04/14.35.33.516796.log (deflated 55%)
  adding: content/.config/logs/2023.01.04/14.35.04.532501.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.03.867433.log (deflated 92%)
  adding: content/.config/logs/2023.01.04/14.35.34.388081.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.56.463059.log (deflated 86%)
  adding: content/.config/logs/2023.01.04/14.34.28.750487.log (deflated 53%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/conf

In [7]:
files.download('/content/sample_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>